In [127]:
import numpy as np
import ast
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

In [128]:
df = pd.read_excel('sample_data.xlsx')

In [129]:
def get_places_category(places):
    places = ast.literal_eval(row.places)
    places_category = []
    
    for idx in range(len(places)):
        category = None
        if places[idx][4] is not None:
            category = places[idx][4][0]
        elif places[idx][1] is not None:
            if len(places[idx][1]) > 1:
                category = places[idx][1]
            else:
                category = places[idx][1][0]
        places_category.append(category)
        
    return places_category

In [130]:
categories_map = pd.DataFrame(columns = ['id', 'category'])
category_set = {}
category_set_inv = {}
category_id = 1

sample = pd.DataFrame(columns=['id', 'places', 'timestamp'])

hashed_mad_id = ""
id = 0
for row in df.itertuples():
    if row.hashed_mad_id != hashed_mad_id:
        id += 1
        hashed_mad_id = row.hashed_mad_id
    if isinstance(row.places, str) == False:
        continue        
    
    places = ast.literal_eval(row.places)
    
    for idx in range(len(places)):
        category = None

        if places[idx][4] is not None:
            category = places[idx][4][0]
        elif places[idx][1] is not None:
            if len(places[idx][1]) > 1:
                category = places[idx][1]
            else:
                category = places[idx][1][0]

        if str(category) in category_set:
            continue
        categories_map = categories_map.append({'id': category_id, 'category': category}, ignore_index=True)
        category_set[str(category)] = category_id
        category_set_inv[str(category_id)] = str(category)
        category_id += 1
    
    sample = sample.append({'id': id, 'places': get_places_category(places), 'timestamp': row.TIMESTAMP},
                           ignore_index=True)

In [131]:
category_set

{'WORK': 1,
 'school': 2,
 'HOME': 3,
 "['hair_care', 'well_being']": 4,
 'bakery': 5,
 "['qsr', 'restaurant']": 6,
 'car_dealer': 7,
 'university': 8,
 'restaurant': 9,
 'well_being': 10,
 "['electronics_store', 'store']": 11,
 "['store', 'home_goods_store']": 12,
 'store': 13,
 'grocery_or_supermarket': 14,
 "['store', 'clothing_store']": 15,
 'library': 16,
 'gas_station': 17,
 "['bar', 'restaurant']": 18}

In [132]:
trajectory_df = pd.DataFrame(columns = ['id', 'trajectory'])

max_idx = id

for i in range(1, max_idx):
    trajectory = list(sample.loc[sample['id'] == i, 'places'])
    trajectory_df = trajectory_df.append({'id': i, 'trajectory': trajectory}, ignore_index = True)

trajectory_df.to_csv('trajectory_df.csv')

In [133]:
categories_map.to_csv('categories_map.csv')

In [134]:
file_content = ""

for row in trajectory_df.itertuples():
    for i in range(len(row.trajectory)):
        trajectory_item = row.trajectory[i]
        for j in range(len(trajectory_item)):
            file_content += str(category_set[str(trajectory_item[j])])
            file_content += " "
        file_content += "-1 "
    file_content += "-2\n"
    
print(file_content)

1 -1 2 -1 1 -1 1 -1 3 -1 4 -1 -2
5 6 -1 6 5 -1 7 -1 8 1 2 -1 8 2 1 -1 9 10 -1 3 -1 2 1 8 -1 3 -1 -2
3 -1 1 8 9 8 2 -1 1 -1 1 -1 3 -1 -2
3 -1 9 1 9 -1 9 1 9 -1 3 -1 3 -1 1 9 9 -1 -2
3 -1 3 -1 11 -1 9 -1 3 -1 12 -1 13 14 13 -1 15 -1 3 -1 -2
8 -1 16 2 2 -1 9 -1 3 -1 2 16 2 9 -1 3 -1 13 -1 1 -1 13 13 -1 3 -1 11 -1 2 16 2 -1 -2
3 -1 17 -1 18 2 -1 14 -1 3 -1 3 -1 3 -1 9 -1 17 -1 -2
3 -1 3 -1 3 -1 -2
1 -1 1 -1 1 -1 3 -1 1 -1 3 -1 -2



In [135]:
file = open("sample.txt", "w")
file.write(file_content)
file.close()

In [136]:
result = open("sample_output.txt", "r") 

result_df = pd.DataFrame(columns=["trajectory", "support", "found_in"])

def strip_list(list_to_strip):
    return list(map(lambda x: x.strip(), list_to_strip))

def build_trajectory_string(trajectory):
    trajectory_string = ""
    has_looped_i = False

    for i in range(len(trajectory) - 1):
        if has_looped_i:
            trajectory_string += " -> "
        has_looped_i = True
        
        itemset = trajectory[i].split(" ")

        has_looped_j = False
        for j in range(len(itemset)):
            if has_looped_j:
                trajectory_string += " | "
            has_looped_j = True
            trajectory_string += category_set_inv[itemset[j]]
    
    return trajectory_string

def build_found_in_string(found_in):
    found_in_string = ""
    has_looped = False

    for i in range(len(found_in)):
        if has_looped:
            found_in_string += ", "
            
        has_looped = True
        found_in_string += str(int(found_in[i]) + 1)
    
    return found_in_string

for line in result:
    splited_on_sup = strip_list(line.split("#SUP:"))
    splited_on_sid = strip_list(splited_on_sup[1].split('#SID:'))
    
    support = splited_on_sid[0].strip()    
    trajectory = strip_list(splited_on_sup[0].split("-1"))
    found_in = strip_list(splited_on_sid[1].split(" "))
    
    trajectory_string = build_trajectory_string(trajectory)
    found_in_string = build_found_in_string(found_in)
    
    result_df = result_df.append({'trajectory': trajectory_string, 'support': support, 'found_in': found_in_string},
                                 ignore_index=True)

result_df.to_csv('result_df.csv')
result_df

,trajectory,support,found_in
0,WORK,6,"1, 2, 3, 4, 6, 9"
1,WORK -> WORK,5,"1, 2, 3, 4, 9"
2,WORK -> WORK -> WORK,5,"1, 2, 3, 4, 9"
3,WORK -> WORK -> WORK -> HOME,4,"1, 2, 3, 9"
4,WORK -> WORK -> HOME,5,"1, 2, 3, 4, 9"
5,WORK -> WORK -> HOME -> WORK,3,"2, 4, 9"
6,WORK -> WORK -> HOME -> HOME,3,"2, 4, 9"
7,WORK -> school,3,"1, 2, 6"
8,WORK -> HOME,6,"1, 2, 3, 4, 6, 9"
9,WORK -> HOME -> WORK,3,"2, 4, 9"
